# Pre-Conference Set-Up
*Becky Hodge*

#### Summary
Every May and November, the Atlanta Writer's Club holds a Writing Conference at the Westin hotel by the Atlanta airport. There are 16-18 agents and editors who offer query letter critiques, manuscript critiques, and hold pitch sessions, two or more speakers who hold workshops throughout the two days on various writing topics, author coaching, and then several other paid events, like the Friday night social, or the morning Q&A panels with editors and agents.

**This code can be run as soon as all the editors and agents are confirmed and announced on the Atlanta Writers Conference website when it opens for registration for the next conference.**

## 1. Steps to set up for the latest conference

Wow, is it time already for the next conference? Didn't we just finish the last one??

Yeps, it's time! And there's a few things to do to finish out the old one and set up for the new one...

#### Close out the prior conference

To close out the old conference, rename the *Outputs* folder to the conference we just left behind (e.g., *May_2025*). 
    
Also, go ahead and duplicate the *Email scheduling dates.xlsx*, *List_of_genres_agents_editors.xlsx*, and *People to hardcode.xlsx* excel files and copy them into that folder - you'll want to make copies because we'll go ahead and reuse those (with any requisite updates) for the next conference.

#### Update the Excel files

There's a couple different excel files that will need updating:

1. *Email scheduling dates.xlsx* --> ou can update this in communication with George. The number of days that each of the tasks should be done prior to the conference should be largely the same.

2. *People to hardcode.xlsx* --> you can just wipe the data in it in preparation for the requests we'll undoubtedly get in the coming months prior to the conference.

3. *List_of_genres_agents_editors.xlsx* --> This one's got the biggeer changes needed now for the rest of the code to work.

    * *fiction* and *nonfiction* --> You can keep these as they are. These shouldn't need to change unless we change these listings on the conference registration form

    * *rooms_fri*, *rooms_sat*, and *timeslots* --> These shouldn't need any updates (and were designed to be that way). **The only exception is if we have fewer or more than 18 editors/agents**. In that case, you'll need to update the *rooms_fri* and *rooms_sat* sheets to drop or add rooms.

    * *Agents_and_editors* --> Update everything in here. Make sure you type the genres exactly as they're found in the 'fiction' and 'nonfiction' sheets, separated by commas. Also, you may want to download the Cvent report to get every agent and editor's names, so they're spelled exactly as they're output - like with any accents, etc.

    * *Coaches* --> Change the name of the coaches. But you can keep everything else the same.

    * *minis_fri* and *minis_sat* --> You should only need to update the designation, speaker and topic columns.
    
    * *timekeepers* --> You can't update this one yet... But that's because we have to run the code below first, so we can get the final pairings of the agents and editors, as well as their room assignments on both days. 

#### Update the templates

First, copy everything in the templates folder and add it to the prior conference folder (it's part of closing out the prior one). 

Then update each of the templates to fix the dates. That should be the only thing you need to change - but make sure to fix it everywhere it's applicable.

## Data Cleaning

#### Load and clean the different files/reports

In [39]:
# Install any needed packages
import pandas as pd
import numpy as np
import datetime
import os 

today = datetime.datetime.today().strftime('%Y-%m-%d')

# Set the conference dates
date_str_fri = '2025-05-02'
date_str_sat = '2025-05-03'


**NOTE:** Before continuing, you need to create a new folder for this latest conference! Go ahead and reference the name below:

In [40]:
current_conference_folder= "May2025"

In [41]:
# Select the file with the most recent date
directory = f'{current_conference_folder}/Cvent_report_downloads'

most_recent_file = max(
    (f for f in os.listdir(directory) if f.startswith('Registered_') and f.endswith('.csv')),
    key=lambda x: datetime.datetime.strptime(x.split('_')[1].split('.')[0], '%m-%d-%y'),
)

# Load the most recent file
most_recent_path = os.path.join(directory, most_recent_file)
registered = pd.read_csv(most_recent_path)


In [42]:
most_recent_file = max(
    (f for f in os.listdir(directory) if f.startswith('Waitlists_') and f.endswith('.csv')),
    key=lambda x: datetime.datetime.strptime(x.split('_')[1].split('.')[0], '%m-%d-%y'),
)

# Load the most recent file
most_recent_path = os.path.join(directory, most_recent_file)
waitlist = pd.read_csv(most_recent_path)

The below code brings in ALL participants, which is key for knowing whether any waitlist only people are virtual or in person.

In [43]:
most_recent_file = max(
    (f for f in os.listdir(directory) if f.startswith('Allparticipants_') and f.endswith('.csv')),
    key=lambda x: datetime.datetime.strptime(x.split('_')[1].split('.')[0], '%m-%d-%y'),
)

# Load the most recent file
most_recent_path = os.path.join(directory, most_recent_file)
all_participants = pd.read_csv(most_recent_path)

In [44]:
all_participants = all_participants.rename(columns={'Email Address':'Email'})

In [45]:
# Filter this dataset to just virtual people
virtual_only = all_participants.loc[all_participants['Hotel vs. Zoom'] == 'Virtually via Zoom (only available for query letter critiques, manuscript sample critiques, and pitches)', :]

In [46]:
del(directory, most_recent_file, most_recent_path)

fict_gen = pd.read_excel(f'{current_conference_folder}/List_of_genres_agents_editors.xlsx', sheet_name='fiction')
nonfict_gen = pd.read_excel(f'{current_conference_folder}/List_of_genres_agents_editors.xlsx', sheet_name='nonfiction')
pubs = pd.read_excel(f'{current_conference_folder}/List_of_genres_agents_editors.xlsx', sheet_name='agents_editors')

Oh gosh, some of the column names are hefty...  Let's fix those.

In [47]:
registered = registered.rename(columns={'Hotel vs. Zoom':'Virtual', 
                                        "What fiction genre(s) will you be presenting to agents/editors at the conference? (If you're not signing up for any agent/editor meetings, indicate which genre(s) you write.)":'Fiction genre', 
                                        "What nonfiction topic(s) will you be presenting to agents/editors at the conference? (If you're not signing up for any agent/editor meetings, indicate which topic(s) you write.)":'Nonfiction genre', 
                                        'Registration Date (GMT)':'Registration Date',
                                        'Email Address':'Email'})

In [48]:
waitlist = waitlist.rename(columns={'Registration Date (GMT)':'Registration Date',
                                     'Email Address':'Email'})

Let's also fix so that we drop the 'Not applicable --I don't write fiction' and 'Not applicable--I don't write nonfiction'. We'll set them to missing.

In [49]:
registered['Fiction genre']= registered['Fiction genre'].replace("Not Applicable --I don't write fiction", np.nan)
registered['Nonfiction genre']= registered['Nonfiction genre'].replace("Not Applicable--I don't write nonfiction", np.nan)

Also, there's people who wrote in 'Other',  but for our purposes, we don't care about that info for the purposes of matching to agents/editors. Let's remove those.

In [50]:
import re
import numpy as np
def clean_genres(genre_string):
    if genre_string is None or pd.isna(genre_string) or "":
        return ""

    genres = [genre.strip() for genre in genre_string.split(',')]
    cleaned_genres = [genre for genre in genres if not re.match(r"^Other \(please specify\):", genre)]

    return ", ".join(cleaned_genres)

registered['Fiction genre'] = registered['Fiction genre'].apply(clean_genres)
registered['Nonfiction genre'] = registered['Nonfiction genre'].apply(clean_genres)


Lastly, let's replace a few of the ones that have ' in them, which make things tricky

In [51]:
registered['Fiction genre'] = registered['Fiction genre'].str.replace("Women’s", "Women's")
registered['Fiction genre'] = registered['Fiction genre'].str.replace("Children’s picture/chapter books", "Children's picture/chapter books")
registered['Nonfiction genre'] = registered['Nonfiction genre'].str.replace("Women’s issues", "Women's issues")


#### Drop any unneeded variables

Let's drop any extraneous variables from the waitlist and registration datasets

In [52]:
waitlist.drop(columns=['Registration Date', 'Invitee Status', 'Action', 'Confirmation Number'],axis=1, inplace=True) # columns are 1, rows are 0

registered.drop(columns=['Agenda Item Type', 'Registration Date', 'Registration Type', 'Action'],axis=1, inplace=True) # columns are 1, rows are 0

#### Create lists with all time-by-room values
We need to pull in the start times for each of the time slots for Friday afternoon (query letter critiques), Saturday morning (manuscript critiques), and Saturday afternoon (pitches) sessions. Without worrying about who our timekeepers are, or which agents are assigned to those rooms, we'll create 3 lists with the times-by-room.

In [53]:
room_fr = pd.read_excel(f'{current_conference_folder}/List_of_genres_agents_editors.xlsx', sheet_name='rooms_friday')
room_sat = pd.read_excel(f'{current_conference_folder}/List_of_genres_agents_editors.xlsx', sheet_name='rooms_sat')
timeslots = pd.read_excel(f'{current_conference_folder}/List_of_genres_agents_editors.xlsx', sheet_name='timeslots')

In [54]:
rooms_friday = room_fr.loc[:, 'day':'room_name']
rooms_saturday = room_sat.loc[:, 'day':'room_name']

Now let's combine the timeslots dataset with the friday and saturday rooms datasets to get the lists we need

In [55]:
tslist_fri = pd.merge(timeslots.loc[(timeslots['day']=='Friday') & (timeslots['day_session']=='Afternoon'), :], rooms_friday, how='outer', on='day')

In [56]:
rooms_coach = pd.read_excel(f'{current_conference_folder}/List_of_genres_agents_editors.xlsx', sheet_name='coaches')

In [57]:
tslist_coach = pd.merge(timeslots.loc[(timeslots['day']=='Friday') & (timeslots['day_session']=='Coaching'), :], rooms_coach, how='outer', on='day')

## Create pairwise agent-editor combos

We need to create a dataset that pairs all agents and editors with each other (18x18), so that we get their combined fiction and non-fiction genre listings. We will need this to ensure that the participants assigned to them for the Friday query letter critiques are a good match.

We also need this to determine which agents should be paired with which agents.

In [58]:
cross_pubs = pd.merge(pubs, pubs, how='cross')

# Drop rows where they cross-reference themselves
cross_pubs = cross_pubs.loc[cross_pubs['lit_guest_name_x'] != cross_pubs['lit_guest_name_y'], :]

# Rename
cross_pubs = cross_pubs.rename(columns={'lit_guest_name_x': 'pubname1', 'lit_guest_type_x': 'pubtype1', 'lit_guest_company_x': 'comp1', 'lit_guest_fiction_x': 'fict1', 'lit_guest_nonfiction_x': 'nonfict1', 'lit_guest_name_y': 'pubname2', 'lit_guest_type_y': 'pubtype2', 'lit_guest_company_y': 'comp2', 'lit_guest_fiction_y': 'fict2', 'lit_guest_nonfiction_y': 'nonfict2'})

Now we need to make two columns that are the full combination of elements in the 'fiction' and 'nonfiction' genre columns.

In [59]:
cross_pubs['combined_fiction'] = cross_pubs.apply(
    lambda row: ', '.join(
        sorted(set(row['fict1'].split(', ') + row['fict2'].split(', ')))
    ),
    axis=1
)

print(cross_pubs['combined_fiction'].head())


1    Coming-of-age, Contemporary, Family saga/drama...
2    Christian, Coming-of-age, Contemporary, Family...
3    Coming-of-age, Contemporary, Fantasy, Humor, L...
4    Coming-of-age, Contemporary, Fantasy, Horror/S...
5    Coming-of-age, Contemporary, Family saga/drama...
Name: combined_fiction, dtype: object


Looks great! We have all the unique fiction genres now. We'll just do the same for nonfiction, though that has some missing values, so the code below adjusts for that (not every agent or editor represents nonfiction).

In [60]:
cross_pubs['combined_nonfiction'] = cross_pubs.apply(
    lambda row: ', '.join(
        sorted(
            set(
                (row['nonfict1'].split(', ') if pd.notna(row['nonfict1']) else []) +
                (row['nonfict2'].split(', ') if pd.notna(row['nonfict2']) else [])
            )
        )
    ) if pd.notna(row['nonfict1']) or pd.notna(row['nonfict2']) else None,
    axis=1
)

print(cross_pubs['combined_nonfiction'].head())

1                                                 None
2    Business/leadership/law, Cooking/food/cookbook...
3    Business/leadership/law, Health/diet/wellness,...
4    Essay collection, Self-help/relationships, Tru...
5    Current events/politics/social commentary, Ess...
Name: combined_nonfiction, dtype: object


Perfect! Now, just to be certain it worked, let's count up the unique fiction and nonfiction genres for each guest, as well as for their combined list.

In [61]:
cross_pubs['num_fict1'] = cross_pubs['fict1'].apply(
    lambda x: len(x.split(', ')) if pd.notna(x) else 0
)
cross_pubs['num_fict2'] = cross_pubs['fict2'].apply(
    lambda x: len(x.split(', ')) if pd.notna(x) else 0
)
cross_pubs['num_nonfict1'] = cross_pubs['nonfict1'].apply(
    lambda x: len(x.split(', ')) if pd.notna(x) else 0
)
cross_pubs['num_nonfict2'] = cross_pubs['nonfict2'].apply(
    lambda x: len(x.split(', ')) if pd.notna(x) else 0
)
cross_pubs['num_combined_fict'] = cross_pubs['combined_fiction'].apply(
    lambda x: len(x.split(', ')) if pd.notna(x) else 0
)
cross_pubs['num_combined_nonfict'] = cross_pubs['combined_nonfiction'].apply(
    lambda x: len(x.split(', ')) if pd.notna(x) else 0
)

print(cross_pubs.loc[0:5, ['num_fict1', 'num_fict2', 'num_combined_fict']].head())

   num_fict1  num_fict2  num_combined_fict
1         14         16                 18
2         14         23                 23
3         14          6                 16
4         14          8                 16
5         14         11                 20


Awesome. This worked great, and now the last bit on this step is to calculate the number of *overlapping* fiction and nonfiction genres per pairing.

In [62]:
def count_overlapping_genres(df):
    df['fict1'] = df['fict1'].str.split(',')
    df['fict2'] = df['fict2'].str.split(',')

    df['fiction_overlap'] = df.apply(lambda row: 
                                  len(set(row['fict1']).intersection(set(row['fict2']))), 
                                  axis=1)
    return df

fiction_overlaps = count_overlapping_genres(cross_pubs)


In [63]:
def count_overlapping_nonfiction(df):

    def clean_genres(genre_list):
        if pd.isna(genre_list) or genre_list == 'None' or not genre_list:
            return []
        else:
            try:
                return genre_list.split(',')
            except AttributeError:
                # Handle cases where genre_list is not a string 
                # (e.g., if it's a list or another object)
                return [] 

    df['nonfict1'] = df['nonfict1'].astype(str).apply(clean_genres) 
    df['nonfict2'] = df['nonfict2'].astype(str).apply(clean_genres) 

    df['nonfiction_overlap'] = df.apply(lambda row: 
                                  len(set(row['nonfict1']).intersection(set(row['nonfict2']))), 
                                  axis=1)
    return df

final_cross_pubs = count_overlapping_nonfiction(fiction_overlaps)

Perfect. Now let's drop any unncessary variables and JUST keep both guests names, their types (agent or editor), and their combined lists of fiction and nonfiction.

In [64]:
del(cross_pubs, fiction_overlaps)
final_cross_pubs = final_cross_pubs.drop(['comp1', 'comp2', 'fict1', 'fict2', 'nonfict1', 'nonfict2', 'num_fict1', 'num_fict2', 'num_nonfict1', 'num_nonfict2'], axis=1)

## Rank publishers

Amazing! Now we've gotten the number of unique genres in each of these combined fiction and nonfiction lists for all pairings of agent-agent, editor-editor, and editor-agent. This will help us more easily identify which agent-editor, editor-editor, and agent-agent pairings make the most sense, in terms of representing the most genres and being able to meet the greatest amount of participant needs for Friday's query letter critiques.

To better help us though, let's rank the pairings. We'll rank pairings according to the number of fiction and nonfiction genres represented between them.

In [65]:
final_cross_pubs[['pubtype1', 'pubtype2']].value_counts()


pubtype1  pubtype2
Agent     Editor      81
Editor    Agent       81
Agent     Agent       72
Editor    Editor      72
Name: count, dtype: int64

In [66]:
final_cross_pubs['rank_fiction'] = final_cross_pubs['num_combined_fict'].rank(ascending=False, method='dense')
final_cross_pubs['rank_nonfiction'] = final_cross_pubs['num_combined_nonfict'].rank(ascending=False, method='dense')

Let's also check to see what each publisher's average ranking is (it's clear some people who represent a ton of genres will average high). Note that this is the sum of a publisher's rankings across all their agent-editor, editor-editor, and agent-agent pairings.

In [67]:
ranks_per_pub = pd.DataFrame(final_cross_pubs.groupby('pubname1')['rank_fiction'].mean().reset_index())

Awesome! Okay, now let's add a few variables:
1) The ratio of total fiction represented across the pairing to the overlap shared between them (# combined fiction genres / # fiction genres overlapping)
2) The ratio of total nonfiction genres represented across the pairing to their overlap
3) The sum of these two ratios

We'll use these to help identify which pairings are best.

In [68]:
final_cross_pubs['ratio_fiction'] = final_cross_pubs['num_combined_fict'].div(final_cross_pubs['fiction_overlap'])
final_cross_pubs['ratio_nonfiction'] = final_cross_pubs['num_combined_nonfict'].div(final_cross_pubs['nonfiction_overlap'])
final_cross_pubs['sum_ratios'] = final_cross_pubs['ratio_fiction'] + final_cross_pubs['ratio_nonfiction']

Great! Now let's just add the avg ranking variable to the final_cross_pubs list.

In [69]:
final_pubs = final_cross_pubs.merge(ranks_per_pub, how= 'outer', on='pubname1')

In [70]:
final_pubs = final_pubs.rename(columns={'rank_fiction_y':'avg_pub_rank',
                                        'rank_fiction_x':'rank_fiction'})
final_cross_pubs = final_pubs
del(final_pubs)

### Pair the publishers for Friday

In this section, we'll use the final_cross_pubs dataset to pair the publishers for Friday's query critiques. We'll do so by starting with the publisher with the highest average ranking (meaning the agent or editor who had the least in common, on average, with all the other editors and agents). 

So for instance, we'll start with the agent/editor who has the LEAST overlap with the others. Then we'll select their five pairings with the lowest ratio sum. This means we'll be selecting the five other agents/editors who had the MOST overlap with them in both fiction and nonfiction genres (anyone with no overlap is being excluded entirely). We'll then sort this list by the publisher type and ratio sum. If the person with the least in common with everyone else is an editor, we then prioritize to pick among the agents first in this top 5 list, and then if there aren't any, we'll pair them with another editor (and vice versa if they're an agent).

Let's set this up.

In [71]:
# Create an empty list to save the pairings
selected_pairs =[]

# We'd like to exclude any same-type pairings, so we don't get agent-agent or editor-editor pairings
df = final_cross_pubs[final_cross_pubs['pubtype1'] != final_cross_pubs['pubtype2']]

# Let's also exclude any rows where the sum_ratio is Infinity - couldn't do this. Meant there weren't rows for certain pairings
#df = final_cross_pubs[np.isfinite(final_cross_pubs['sum_ratios'])]

In [72]:
top_pubname = df.loc[df['avg_pub_rank'].idxmax(), 'pubname1']

# Filter rows for this pubname1
filtered = df[df['pubname1'] == top_pubname]

pubtype1 = filtered.iloc[0]['pubtype1']
if pubtype1 == 'Editor':
    filtered = filtered.sort_values(
        by=['pubtype2', 'sum_ratios'], ascending=[True, True])
else:  # pubtype1 == 'Agent'
    filtered = filtered.sort_values(
        by=['pubtype2', 'sum_ratios'], ascending=[False, True])

check = filtered[['pubname1', 'pubtype1', 'pubname2', 'pubtype2', 'avg_pub_rank', 'sum_ratios']]
check2 = check.sort_values(by=['pubtype2', 'sum_ratios'], ascending=[False, True])

In [73]:
while not df.empty:
    # Identify the publisher with the highest rank (regardless of whether they are pubname1 or pubname2)
    top_pubname = df.loc[df['avg_pub_rank'].idxmax(), 'pubname1']

    # Filter rows for this pubname1
    filtered = df[df['pubname1'] == top_pubname]

    # Sort these rows dynamically based on `pubtype1` and `pubtype2`
    pubtype1 = filtered.iloc[0]['pubtype1']
    if pubtype1 == 'Editor':
        filtered = filtered.sort_values(by=['pubtype2', 'sum_ratios'], ascending=[True, True])
    if pubtype1 == 'Agent':
        filtered = filtered.sort_values(by=['pubtype2', 'sum_ratios'], ascending=[False, True])

    # Select the first row from the sorted list
    selected_row = filtered.iloc[0]
    selected_pairs.append(selected_row)

    # Drop all rows containing the selected `pubname1` or `pubname2`
    df = df[~df['pubname1'].isin([selected_row['pubname1'], selected_row['pubname2']]) &
            ~df['pubname2'].isin([selected_row['pubname1'], selected_row['pubname2']])]

    # Stop if we have 9 rows in the result
    if len(selected_pairs) == 9:
        break

# Step 4: Create a new DataFrame or list with the selected rows
result_df = pd.DataFrame(selected_pairs)
result_df = result_df[['pubname1', 'pubtype1', 'pubname2', 'pubtype2']]

# Print or save the result
print(result_df)

             pubname1 pubtype1             pubname2 pubtype2
104  Joëlle Delbourgo    Agent         Kurt Brackob   Editor
76        Jake Lovell    Agent            Grace Gay   Editor
187     Micah Brocker    Agent  Foyinsi Adegbonmire   Editor
145       Jéla Lewter   Editor      Paloma Hernando    Agent
6    Alexandria Brown   Editor       Renée Fountain    Agent
133   Jynastie Wilson    Agent          Dianna Vega   Editor
216  Monica Rae Brown   Editor  Jenna Satterthwaite    Agent
181     Lauren Bieker    Agent        Nicole Luongo   Editor
288       Vicky Weber    Agent           Wendy Wong   Editor


In [74]:
# Confirm we have 9 pairings
len(result_df)

9

In [75]:
# Now sure why this is happening, but we need to remove white spaces at teh beginning of the series
final_cross_pubs['combined_fiction'] = final_cross_pubs['combined_fiction'].str.lstrip()
final_cross_pubs['combined_nonfiction'] = final_cross_pubs['combined_nonfiction'].str.lstrip()

In [78]:
# Now filter to the editor-agent pairs that we actually have, instead of keeping EVERY combo:
final_cross_pubs = pd.merge(final_cross_pubs, final_room_pairings_Friday, how="inner")

### Assign the paired publishers their rooms

Woohoo! These all look good and make sense, so we're good to go for the room pairings for Friday's query letter critiques. Let's just assign these pairings to actual rooms now.

In [76]:
final_room_pairings_Friday = pd.concat([room_fr.reset_index(drop=True), result_df.reset_index(drop=True)], axis=1)

Amazing!! We are now officially done with determining Friday's pairings. Let's just save this info:

In [79]:
final_room_pairings_Friday.to_excel(f"{current_conference_folder}/Outputs/Finalized Datasets/Editor-agent pairings for Friday.xlsx", index=False)
final_cross_pubs.to_excel(f"{current_conference_folder}/Outputs/Finalized Datasets/Final editor-agent pairings with combined genres.xlsx", index=False)

Lastly, before we finish this tidbit of code, **make sure to note which agents and editors are in which rooms for Saturday**. You'll want to put this in the *timekeeper* sheet of the *List_of_agents_editors.xlsx*, along with the info on rooms and agent pairings for Friday, since we'll need both of these and will want to ensure that every publisher or publisher pairing on both days has an assigned timekeeper.

This info won't change (it'll always be coded the same), and none of the code in this entire program will result in different output either, so feel free to rerun this code later if you forgot this step.

In [44]:
print(rooms_saturday)

         day       room_name
0   Saturday    Board Room I
1   Saturday   Board Room II
2   Saturday  Board Room III
3   Saturday   Board Room IV
4   Saturday    Board Room V
5   Saturday   Board Room VI
6   Saturday    Fayetteville
7   Saturday       Riverdale
8   Saturday         Dogwood
9   Saturday        Gardenia
10  Saturday         Jasmine
11  Saturday        Magnolia
12  Saturday   Cherokee Rose
13  Saturday       Atlanta I
14  Saturday      Atlanta II
15  Saturday     Atlanta III
16  Saturday      Atlanta IV
17  Saturday  Peachtree City


You note down the above??

Perfect! No need to run any more code for a little while. The other sections of code don't need to be run until ~1 month before the conference.

NOTE: We can potentially integrate with Cvent and pull in data directly via SQL queries. Follow this code: https://www.cdata.com/kb/tech/cvent-python-pandas.rst